# __Web Scraping desde la paractica: Paginas Dinamicas__

Las paginas web dinamicas renderizan el HTML conforme vamos haciendo scroll de la misma. Esto genera un gran desafio a la hora de scrapear ya que no tiene definido un limite. Para solucionarlo, vamos a usar Selenium.

## Automatizacion de sitios web dinamicos

¿Estás cansado de realizar tareas repetitivas como llenar múltiples veces un mismo formulario o extraer elementos manualmente de un sitio web? La automatización es justamente la herramienta que necesitas para simplificar estos procesos rutinarios.

En lo que sigue, vamos a obtener el HTML renderizado de una pagina dinamica y hacer ver algunas dificultades que presentan este tipo de paginas. La dificultad radica en que estos sitios web dinámicos no te permiten acceder directamente al contenido generado por el JavaScript. Al realizar peticiones GET con la librería Requests, solo obtienes JavaScript y elementos HTML estáticos, lo cual limita tu capacidad para extraer contenidos específicos que aparecen al hacer scroll.

### Importar librerias

In [22]:
import requests

### Obtener sitio web dinamico desde una peticion

In [23]:
# definir url
url = 'https://quotes.toscrape.com/scroll'

# realizar la peticion GET
website = requests.get(url)

# mostrar la web en texto
print(website.text)

<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<title>Quotes to Scrape</title>
    <link rel="stylesheet" href="/static/bootstrap.min.css">
    <link rel="stylesheet" href="/static/main.css">
    
    
</head>
<body>
    <div class="container">
        <div class="row header-box">
            <div class="col-md-8">
                <h1>
                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>
                </h1>
            </div>
            <div class="col-md-4">
                <p>
                
                    <a href="/login">Login</a>
                
                </p>
            </div>
        </div>
    
<div class="row">
    <div class="col-md-8">
        <div class="quotes"></div>
    </div>
</div>
<div id="loading" style="background-color: #eeeecc"><h5>Loading...</h5></div>
<script src="/static/jquery.js"></script>
<script>
    $(function(){
        var page = 1, tag = null, hasNextPage = true;
        function appendQu

> El HTML de la pagina presenta un codigo en javascript que permite que aparezcan los elementos a medida que se scrollea. Sin embargo, esto limita la capacidad de scrapear con los metodos antes utilizados

## Creacion de Scraper con Selenium

Crear un scraper eficaz usando Selenium y BeautifulSoup permite obtener información actualizada de páginas dinámicas con facilidad. Aprende cómo configurar la librería Selenium con WebDriver, acceder a una URL específica, gestionar el renderizado del HTML y extraer datos útiles como frases y autores.

### Dependencias

In [24]:
import time
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

### Configuracion del Servidor y Driver 

In [25]:
# ruta al chromedriver
chrome_driver_path = '/usr/local/bin/chromedriver-linux64/chromedriver'

# configurar opciones para visualizar el paso a paso de la interaccion con el navegador
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # pantalla completa
options.add_argument('--disable-blink-features=AutomationControlled') # menos detectable

# Crear el servidor y el driver
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)


### Definir url

In [26]:
# definir url
url = 'https://quotes.toscrape.com/scroll'

# realizar peticion GET
driver.get(url)

# definir un tiempo de espera para que se renderice el HTML
time.sleep(3)

### Manipular HTML renderizado

In [27]:
# obtener HTML renderizado
html = driver.page_source
driver.quit()

# procesar el HTML con beautifulSoup
soup = BeautifulSoup(html, 'html.parser')
quotes = soup.select('div.quote')

print('Citas encontradas: ', len(quotes))

for quote in quotes:
    text = quote.find('span', class_='text').get_text()
    autor = quote.find('small', class_='author').get_text()
    print(f'{text} - {autor}')

Citas encontradas:  10
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” - Albert Einstein
“It is our choices, Harry, that show what we truly are, far more than our abilities.” - J.K. Rowling
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” - Albert Einstein
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” - Jane Austen
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” - Marilyn Monroe
“Try not to become a man of success. Rather become a man of value.” - Albert Einstein
“It is better to be hated for what you are than to be loved for what you are not.” - André Gide
“I have not failed. I've just found 10,000 ways that won't work.” - Thomas A. Edison
“A woman is like a tea bag; you never know how strong it is until it's in hot

## Automatizacion de scroll

Para realizar un scraping eficaz en sitios web que cuentan con scroll infinito es fundamental implementar correctamente las interacciones con el desplazamiento (scroll) de la página web. 

### Dependencias

In [28]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

### Configuracion del servicio y driver

In [29]:
# ruta al chromedriver
chrome_driver_path = '/usr/local/bin/chromedriver-linux64/chromedriver'

# configurar opciones para visualizar el paso a paso de la interaccion con el navegador
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # pantalla completa
options.add_argument('--disable-blink-features=AutomationControlled') # menos detectable

# Crear el servidor y el driver
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)


### Definir URL

In [30]:
# definir url
url = 'https://quotes.toscrape.com/scroll'

# realizar peticion GET
driver.get(url)

### Implementacion

In [31]:
# definir tiempo para realizar scroll
# dado en segundos
scroll_pause_time = 2

# define la altura del scroll
last_height = driver.execute_script('return document.body.scrollHeight')
quotes_set = set()

# desplazarse varias veces por la pagina (realiza 3 scroll)
for i in range(3):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(scroll_pause_time) # configura el tiempo de acuerdo a lo que tarda la pagina en renderizar sus elementos
    new_height = driver.execute_script('return document.body.scrollHeight')

    # extraer los elementos de interes
    quotes = driver.find_elements(By.CLASS_NAME, 'quote')
    for quote in quotes:
        text = quote.find_element(By.CLASS_NAME, 'text').text
        quotes_set.add(text)

    # detener proceso en caso que el scroll se termine
    if new_height == last_height:
        break

    # actualizar la altura
    last_height = new_height

driver.quit()


In [33]:
# mostrar cantidad y frases obtenidas
print(f'Total de frases unicas cargadas: {len(quotes_set)}')
for quote in quotes_set:
    print(quote)

Total de frases unicas cargadas: 40
“The more that you read, the more things you will know. The more that you learn, the more places you'll go.”
“The opposite of love is not hate, it's indifference. The opposite of art is not ugliness, it's indifference. The opposite of faith is not heresy, it's indifference. And the opposite of life is not death, it's indifference.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good part is you get to decide how you're going to mess it up. Girls will be your friends - they'll act like it anyway. But just remember, some come, some go. The ones that stay with you through everything - they're your true best friends. Don't let go of them. Also remember, sisters make the best friends in the world. As for lovers, well, they'll come and go too. And baby, I hate